In [1]:
import os
import tensorflow as tf
import sys
import math
import cv2
import imutils
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.backend import resize_images
from tensorflow.keras.models import load_model

In [7]:
model_all = load_model('/home/venkat/Downloads/trial_all_class_weight.h5')
model_correct = load_model('/home/venkat/Downloads/trial_correct.h5')
model_curve = load_model('/home/venkat/Downloads/trial_curve.h5')
model_fingers_up = load_model('/home/venkat/Downloads/trial_fingers_up.h5')
model_fist = load_model('/home/venkat/Downloads/trial_fist.h5')
model_lkxy = load_model('/home/venkat/Downloads/trial_lkxy.h5')
model_point = load_model('/home/venkat/Downloads/trial_point.h5')
print("Model loaded!")
branch = {}
branch[0] = model_fist
branch[1] = model_fingers_up
branch[2] = model_curve
branch[3] = model_fingers_up
branch[4] = model_fist
branch[5] = model_correct
branch[6] = model_point
branch[7] = model_point
branch[8] = model_fingers_up
branch[10] = model_lkxy
branch[11] = model_lkxy
branch[12] = model_fist
branch[13] = model_fist
branch[14] = model_curve
branch[15] = model_correct
branch[16] = model_correct
branch[17] = model_fingers_up
branch[18] = model_fist
branch[19] = model_point
branch[20] = model_fingers_up
branch[21] = model_correct
branch[22] = model_correct
branch[23] = model_lkxy
branch[24] = model_lkxy
print("Branch models added")

Model loaded!
Branch models added


In [9]:
videoCaptureObject = cv2.VideoCapture(0)
i = 0
ret = True

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (480,640))
start_point = (50,100)
end_point = (300, 350) 
c = 0
# l=[]
po = np.zeros((25,1))

while(ret):
    ret,frame = videoCaptureObject.read()
    if ret == True:
        clone = frame.copy() 
        frame = cv2.rectangle(frame, start_point, end_point, color=(0,0,255), thickness=2)
        
        if c%5==0:
            crop_img = clone[100:350,50:300]
            resized_img = cv2.resize(crop_img,(100,100))

            img = cv2.cvtColor(resized_img ,cv2.COLOR_BGR2YUV)
            img[:,:,0] = cv2.equalizeHist(img[:,:,0])
            img_hist = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
            denoised = cv2.bilateralFilter(img_hist, 7, 100, 100) 
            v = np.median(denoised)
            lower = int(max(0, (1.0 - 0.33) * v))
            upper = int(min(255, (1.0 + 0.33) * v))
            img_blur_canny = cv2.Canny(denoised,lower,upper)
            
            x_test = np.asarray(denoised)
            x_test = x_test.astype('float32')
            y_pred = model_all.predict(x_test.reshape((1,100,100,3)), batch_size=None, steps=1)
            display_text = ""
            max_pred = np.max(y_pred)
            if(max_pred>0.5):
                index = np.argmax(y_pred)
                model_temp = branch[index]
                y_branch = model_temp.predict(x_test.reshape((1,100,100,3)), batch_size=None, steps=1)
                display_text = "Alphabet - " + chr(ord('A') + np.argmax(y_branch))
            else:
                display_text = "Need better picture"
        c += 1  
        frame = cv2.flip(frame,1)
        cv2.putText(frame, display_text, (340, 95), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 3)
        cv2.imshow('ASL Alphabet Classification', frame)
        # doesn't work with colab (cv2.imshow)
        
        
        out.write(frame)
        pressedKey = cv2.waitKey(1) & 0xFF
        if(pressedKey == ord('c')):
            
            cv2.imwrite("A/A"+ str(i)+".jpg",resized_img)
            plt.imshow(resized_img, vmin=0, vmax=255)
            plt.show()
            i += 1
        elif(pressedKey == ord('q')):
            videoCaptureObject.release()
            out.release()
            cv2.destroyAllWindows()
    else:
        print("Happy Pictures Taken!")


Happy Pictures Taken!
